In [32]:
using MatrixNetworks
using Distributions
using Gadfly
using Clustering
using BenchmarkTools, Compat
using DelimitedFiles
using SparseArrays
using LinearAlgebra
using SphericalGraphEmbedding
using Random

In [34]:
dataset = "PolBlogs"
dataset = "PowerEU"
dataset = "Facebook"
dataset = "PowerUS"
dataset = "HEP"
dataset = "Internet"
dataset = "CondMat2003"
#dataset = "Pokec"

# Initialization
A = 0
r = 0
dim0 = 0
community = zeros(Int64,N,1);

# nb iterations projected power method
n_it_PPM = 30000

# relative objective variation
t = 1e-10

# nb of times vector partition is repeated
n_it_vec_part = 100

# nb updates vec part
n_updates = 100 

# shape of embedding
shape = "Ellipsoidal"

# Loading data

if dataset == "PolBlogs"

    A0 = readdlm("data/PolBlogs/PolBlogsAdj.txt")
    lab = readdlm("data/PolBlogs/LabelsPolBlogs.txt")

    dim0 = size(A0)
    A  = zeros(Int64 ,dim0[1],dim0[1])

    for i = 1:dim0[1]
        for j = 1:dim0[1]
            A[i,j] = Int64(A0[i,j])
        end
    end
    A = A+A'
    A = sparse(A)

    A,p = largest_component(A)
    lab = lab[p]
    A0 = 0
    p = 0 
    N = size(A,2)

    # number of columns of initial guess
    r = 20
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "PowerEU"

    list = readdlm("data/PowerEU/ElistPowerEU.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j])
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])])

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,p = largest_component(A0)

    A0 = 0
    p = 0
    m = 0

    # number of columns of initial guess
    r = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "Facebook"

    list = readdlm("data/Facebook/facebook-gender_edges.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) + 1 # here start at 0
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]) # here start at 0

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,p = largest_component(A0)

    A0 = 0
    p = 0
    m = 0

    # number of columns of initial guess
    r = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "PowerUS"

    list = readdlm("data/PowerUS/ElistPowerUS.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j])
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])])

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,p = largest_component(A0)

    A0 = 0
    p = 0
    m = 0

    # number of columns of initial guess
    r = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "HEP"

    list = readdlm("data/HEP/hep-year_edges.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) 
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]) 

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,p = largest_component(A0)

    A0 = 0
    p = 0
    m = 0

    # number of columns of initial guess
    r = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "Wikivote"

    list = readdlm("data/Wikivote/wiki-Vote.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) 
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]) 

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,p = largest_component(A0)

    A0 = 0
    p = 0
    m = 0

    # number of columns of initial guess
    r = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "Internet"

    list = readdlm("data/Internet/InternetEList.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) + 1 # index start from 0
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])]) 

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,p = largest_component(A0)

    A0 = 0
    p = 0
    m = 0

    # number of columns of initial guess
    r = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "CondMat2003"

    list = readdlm("data/CondMat2003/CondMat2003EList.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) + 1 # index start from 0
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])])

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,p = largest_component(A0)

    A0 = 0
    p = 0
    m = 0

    # number of columns of initial guess
    r = 50
    # number of centroids thrown in embedding
    n_clusters = 100

elseif dataset == "Pokec"

    list = readdlm("data/Pokec/soc-pokec-relationships.txt")
    dim0 = size(list)

    M_int = zeros(Int64,dim0[1],2)

    for i = 1:dim0[1]
        for j = 1:2
            M_int[i,j] = Int64(list[i,j]) + 1 # index start from 0
        end
    end

    N = maximum([maximum(M_int[:,1]),maximum(M_int[:,2])])

    A0 = spzeros(N,N)
    A0 = sparse(M_int[:,1],M_int[:,2],vec(ones(Int64,dim0[1],1)),N,N)
    A0 = A0 + A0'
    A,p = largest_component(A0)

    A0 = 0
    p = 0
    m = 0

    # number of columns of initial guess
    r = 50
    # number of centroids thrown in embedding
    n_clusters = 100

end

CondMat2003

In [35]:
print(dataset)
x_embed,community,singular =  @time sphere_embed_cluster(A, n_it_PPM, t, n_clusters, n_it_vec_part, n_updates, shape, r)
# run twice to get timings without compilation time

 ------- Acc Projected Power Iteration -------
The iteration has become stationary after 290 iterations
 32.632579 seconds (8.58 k allocations: 26.811 GiB, 12.70% gc time)
 -------------- Clustering ------- 
dimension of embedding used for clustering: 50
Number of updates: 21
Number of communities: 11
Modularity: 0.6205025895121782
 -------------------------------------------- 
The first 5 squared singular values divided by N : 
[0.6105938822972591, 0.14174232371652565, 0.12040875024145256, 0.07550560512521615, 0.029506047492906355]
 -------------------------------------------- 
522.724052 seconds (129.68 M allocations: 125.492 GiB, 5.37% gc time)


([0.0044560657114592565 0.00617495920059514 … -0.0072735455118215905 -0.007193259986357211; 0.009319016842885672 0.005720856449299675 … -0.005281275448439841 -0.0033338312391468943; … ; -0.0036459923892575652 -0.0033687470230055277 … -0.0065961534286518375 -0.003332344222272281; -0.0031457641171445854 -0.001103004326906282 … -0.0014791791626605389 -0.002854125486054118], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  2, 2, 2, 2, 2, 2, 2, 2, 2, 2])